# Crawling et utilisation de l'API Github

INFMDI721 - TP - 16/10/2019

Consigne:
- Récupérer via crawling la liste des 256 top contributors sur une journée sur cette page https://gist.github.com/paulmillr/2657075 ;
- En utilisant l'API github (https://developer.github.com/v3/) récupérer pour chacun de ces users le nombre moyens de stars des repositories qui leur appartiennent. Pour finir, classer ces 256 contributors par leur note moyenne.

Note:
En requêtant les repos d'un utilisateur sur l'API Github, il faudrait prendre en compte la pagination des résultats renvoyés

In [4]:
import requests
from bs4 import BeautifulSoup
from github import Github # install PyGithub - doc : https://pygithub.readthedocs.io
import pandas as pd

## Récupération de la liste des utilisateurs

In [7]:
page = "https://gist.github.com/paulmillr/2657075"
response = requests.get(page)
soup = BeautifulSoup(response.text)

In [8]:
rows = soup.find("tbody").find_all("tr")

In [9]:
users = []
for row in rows:
    users.append(row.find("a").text)
print(users)

['fabpot', 'andrew', 'taylorotwell', 'egoist', 'HugoGiraudel', 'ornicar', 'bebraw', 'nelsonic', 'alexcrichton', 'jonathanong', 'mikermcneil', 'benbalter', 'jxnblk', 'yegor256', 'orta', 'rstacruz', 'GrahamCampbell', 'afc163', 'kamranahmedse', 'joshaber', 'bkeepers', 'kennethreitz', 'kytrinyx', 'STRML', 'atmos', 'weierophinney', 'agentzh', 'steipete', 'ai', 'mikepenz', 'nvie', 'hadley', 'appleboy', 'Rich-Harris', 'drnic', 'rafaelfranca', 'Ocramius', 'mitchellh', 'stof', 'IgorMinar', 'phodal', 'jwiegley', 'geerlingguy', 'dcramer', 'sebastianbergmann', 'brunocvcunha', 'ljharb', 'jdalton', 'sevilayha', 'paulmillr', 'tmm1', 'c9s', 'zcbenz', 'holman', 'kevinsawicki', 'yihui', 'buckyroberts', 'kbrsh', 'dmalan', 'mhevery', 'mgechev', 'kylef', 'chjj', 'ayende', 'mcollina', 'mdo', 'yoshuawuyts', 'muan', 'kentcdodds', 'jskeet', 'mitsuhiko', 'steveklabnik', 'hzoo', 'Caged', 'dlew', 'technoweenie', 'gaearon', 'soumith', 'feross', 'michalbe', 'brianleroux', 'willdurand', 'alexjlockwood', 'matsko', 's

## Connexion à l'API Github

In [10]:
import json
with open("credentials.json") as f:
    data = json.load(f)
    token = data['token']

In [11]:
g = Github(token)

## Récupération des informations sur les utilisateurs

In [12]:
df = pd.DataFrame(columns=["user", "nb_repos", "mean_stars"])

In [13]:
for user in users:
    print(user)
    user_infos = g.get_user(user)
    count_repos = 0
    count_stars = 0
    for repo in user_infos.get_repos():
        if not repo.fork:
            count_repos += 1
            count_stars += repo.stargazers_count
    df = df.append({"user": user, "nb_repos": count_repos, "mean_stars": round(count_stars / count_repos)}, ignore_index=True)

fabpot
andrew
taylorotwell
egoist
HugoGiraudel
ornicar
bebraw
nelsonic
alexcrichton
jonathanong
mikermcneil
benbalter
jxnblk
yegor256
orta
rstacruz
GrahamCampbell
afc163
kamranahmedse
joshaber
bkeepers
kennethreitz
kytrinyx
STRML
atmos
weierophinney
agentzh
steipete
ai
mikepenz
nvie
hadley
appleboy
Rich-Harris
drnic
rafaelfranca
Ocramius
mitchellh
stof
IgorMinar
phodal
jwiegley
geerlingguy
dcramer
sebastianbergmann
brunocvcunha
ljharb
jdalton
sevilayha
paulmillr
tmm1
c9s
zcbenz
holman
kevinsawicki
yihui
buckyroberts
kbrsh
dmalan
mhevery
mgechev
kylef
chjj
ayende
mcollina
mdo
yoshuawuyts
muan
kentcdodds
jskeet
mitsuhiko
steveklabnik
hzoo
Caged
dlew
technoweenie
gaearon
soumith
feross
michalbe
brianleroux
willdurand
alexjlockwood
matsko
stefanpenner
adamwathan
Haacked
curran
rauchg
ianstormtaylor
KrauseFx
tj
jgm
jverkoey
chenglou
DataTables
SamyPesse
mjhea0
0x00A
tmcw
brentvatne
carpedm20
benjamn
notwaldorf
miyagawa
rnystrom
photonstorm
mattn
dougwilson


ReadTimeout: HTTPSConnectionPool(host='api.github.com', port=443): Read timed out. (read timeout=15)

In [14]:
df.head(5)

,user,nb_repos,mean_stars
0,fabpot,3,71
1,andrew,106,9
2,taylorotwell,1,22
3,egoist,622,62
4,HugoGiraudel,36,188


## Tri du dataset par nombre moyen d'étoiles

In [15]:
df = df.sort_values(by=["mean_stars"], ascending=False)
df.head()

,user,nb_repos,mean_stars
18,kamranahmedse,34,4110
44,sebastianbergmann,64,1814
29,mikepenz,19,1317
106,photonstorm,28,1105
65,mdo,21,917
